# Crop simulation

Doing imports 

In [22]:
%matplotlib inline
#%matplotlib notebook
import sys
import pcse
import pandas as pd
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
print("This notebook was built with:")
print(f"python version: {sys.version}")
print(f"PCSE version: {pcse.__version__}")

This notebook was built with:
python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
PCSE version: 6.0.9


## Weather & Solar data 

### 1. Transform the data into the right units


| Identifier | Parameter   | Description                              | From Unit  | To Unit           | Dataset |
|------------|-------------|------------------------------------------|------------|-------------------|---------|
| TXK        | TMAX        | Daily maximum temperature                | °C         | °C                | KL      |
| TNK        | TMIN        | Daily minimum temperature                | °C         | °C                | KL      |
| VPM        | VAP         | Mean daily vapour pressure               | hPa        | hPa               | KL      |
| FM         | WIND        | Mean daily wind speed at 2 m above ground level | m/s        | m/s       | KL      |
| RSK        | RAIN        | Precipitation                            | mm         | cm   | KL      |
| FG_STRAHL  | IRRAD       | Daily global radiation                   | J/cm^2     | J/m^{-2} | Solar   |
| SHK_TAG    | SNOWDEPTH   | Depth of snow cover (optional)           | cm         | cm                | KL      |

##### Climate data

In [41]:
# Read the text file into a DataFrame
data_bremen_kl = pd.read_csv('data_kl/bremen_daily_kl/tageswerte_KL_00691_18900101_20231231_hist/produkt_klima_tag_18900101_20231231_00691.txt', delimiter=';', skipfooter=1, engine='python')

data_chemnitz_kl = pd.read_csv('data_kl/chemnitz_daily_kl/tageswerte_KL_00853_18820101_20231231_hist/produkt_klima_tag_18820101_20231231_00853.txt', delimiter=';', skipfooter=1, engine='python')

# Convert units -> mm to cm, ensuring column names are stripped of extra spaces
data_bremen_kl.columns = data_bremen_kl.columns.str.strip()
data_chemnitz_kl.columns = data_chemnitz_kl.columns.str.strip()

# Replace -999 with NaN
data_bremen_kl.replace(-999, np.nan, inplace=True)
data_chemnitz_kl.replace(-999, np.nan, inplace=True)


##### Solar data

In [ ]:
# Read the text file into a DataFrame
data_bremen_solar = pd.read_csv('data_solar/bremen/tageswerte_ST_00691_row/produkt_st_tag_20090101_20250331_00691.txt', delimiter=';', skipfooter=1, engine='python')

data_chemnitz_solar = pd.read_csv('data_solar/chemnitz/tageswerte_ST_00853_row/produkt_st_tag_19810101_20250331_00853.txt', delimiter=';', skipfooter=1, engine='python')

# drop the 'eor' column from the DataFrame
#data_bremen_solar.drop(columns=['eor'], inplace=True)

# Replace -999 with NaN
data_bremen_solar.replace(-999, np.nan, inplace=True)
data_chemnitz_solar.replace(-999, np.nan, inplace=True)

##### Find the rows without NaNs


In [31]:
# Create a boolean mask for rows without NaN values
mask_bremen = data_bremen_solar.notna().all(axis=1)
mask_chemnitz = data_chemnitz_solar.notna().all(axis=1)

# Filter and print the rows without NaN values
data_bremen_solar_cleaned = data_bremen_solar[mask_bremen]
data_chemnitz_solar_cleaned = data_chemnitz_solar[mask_chemnitz]

print("Bremen Solar Data without NaNs:")
print(data_bremen_solar_cleaned)

print("Chemnitz Solar Data without NaNs:")
print(data_chemnitz_solar_cleaned)

Bremen Solar Data without NaNs:
Empty DataFrame
Columns: [STATIONS_ID, MESS_DATUM, QN_592, ATMO_STRAHL, FD_STRAHL, FG_STRAHL, SD_STRAHL]
Index: []
Chemnitz Solar Data without NaNs:
Empty DataFrame
Columns: [STATIONS_ID, MESS_DATUM, QN_592, ATMO_STRAHL, FD_STRAHL, FG_STRAHL, SD_STRAHL, eor]
Index: []


### 2. Transform to PCSE data format

In [38]:
# Map and transform the data to PCSE format
data_bremen_pcse = pd.DataFrame({
    #'DAY': pd.to_datetime(data_bremen_kl['MESS_DATUM'], format='%Y%m%d'),
    'DAY': data_bremen_kl['MESS_DATUM'],
    'TMIN': data_bremen_kl['TNK'],  # Example: map to minimum temperature
    'TMAX': data_bremen_kl['TXK'],  # Example: map to maximum temperature
    'VAP': data_bremen_kl['VPM'],  # Example: map to vapor pressure
    'RAIN': data_bremen_kl['RSK'] / 10,  # Convert mm to cm if needed
    'IRRAD': data_bremen_solar['FG_STRAHL'] * 10000,  # Convert J/cm² to J/m²
    'WIND': data_bremen_kl['FM'],  # Assuming this is the wind speed
})

#print(data_bremen_pcse)

In [ ]:
# Map and transform the data to PCSE format
data_chemnitz_pcse = pd.DataFrame({
    #'DAY': pd.to_datetime(data_chemnitz_kl['MESS_DATUM'], format='%Y%m%d'),
    'DAY': data_chemnitz_kl['MESS_DATUM'],
    'TMIN': data_chemnitz_kl['TNK'],  # Example: map to minimum temperature
    'TMAX': data_chemnitz_kl['TXK'],  # Example: map to maximum temperature
    'VAP': data_chemnitz_kl['VPM'],  # Example: map to vapor pressure
    'RAIN': data_chemnitz_kl['RSK'] / 10,  # Convert mm to cm if needed
    'IRRAD': data_chemnitz_solar['FG_STRAHL'] * 10000,  # Convert J/cm² to J/m²
    'WIND': data_chemnitz_kl['FM'],  # Assuming this is the wind speed
})

#print(data_chemnitz_pcse)


##### Tests to find error 

In [ ]:
# Remove rows with any NaN values
# data_cleaned = data_chemnitz_pcse.dropna()
# print(data_cleaned)


# print the rows with NaN values & sum rows with NaN values
# print(data_chemnitz_pcse.isna().sum())
# print(data_chemnitz_pcse.isna())

# Filter the data for a specific date range (e.g., from 2009-01-01 to 2023-12-31)
# data_chemnitz_filtered = data_chemnitz_pcse[data_chemnitz_pcse['DAY'] >= 20090101]
# data_chemnitz_filtered.to_csv('data_chemnitz_filtered.csv', index=False, na_rep='NaN')


# Write to CSV file without site characteristics
# data_chemnitz_pcse.to_csv('data_chemnitz_pcse.csv', index=False, na_rep='NaN')

### 3. Insert site characteristics into the CSV

In [ ]:
# Site characteristics as a string
site_characteristics = """## Site Characteristics
Country     = 'Germany'
Station     = 'Chemnitz'
Description = 'Observed data from Station Chemnitz'
Source      = 'Your Data Source'
Contact     = 'Your Contact'
Longitude = 12.92; Latitude = 50.83; Elevation = 300; AngstromA = 0.25; AngstromB = 0.50; HasSunshine = False
## Daily weather observations (missing values are NaN)
"""

# Convert DataFrame to CSV string
csv_data = data_chemnitz_pcse.to_csv(index=False, na_rep='NaN')

# Combine site characteristics and CSV data
full_csv_content = site_characteristics + csv_data

# Write to CSV file
with open('chemnitz_withMeta.csv', 'w') as f:
    f1.write(full_csv_content)
    #f.write(csv_data)

print("CSV file with site characteristics created.")

CSV file with site characteristics created.


### 4. Create and print the WeatherDataProvider

In [49]:
from pcse.input import CSVWeatherDataProvider

# Load the weather data provider
weatherfile = "chemnitz_withMeta.csv"
weatherdataprovider = CSVWeatherDataProvider(weatherfile, delimiter=',', dateformat='%Y%m%d', ETmodel='PM', force_reload=False)

# Debugging: Inspect the first and last date
print(f"First date: {weatherdataprovider.first_date}")
print(f"Last date: {weatherdataprovider.last_date}")

print(weatherdataprovider)

First date: None
Last date: None


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

## Crop, Soil and Agro Data

In [ ]:
from pcse.input import YAMLCropDataProvider
from pcse.models import Wofost72_PP
from pcse.util import DummySoilDataProvider
from pcse.input import WOFOST72SiteDataProvider
from pcse.base import ParameterProvider
from pcse.input import YAMLAgroManagementReader

"""load crop file https://github.com/ajwdewit/WOFOST_crop_parameters"""
cropd = YAMLCropDataProvider(Wofost72_PP) #https://github.com/ajwdewit/WOFOST_crop_parameters
#cropd.print_crops_varieties() #every kind of plant in this library

#activate a specific crop (crop and variety), loaded directly from the PCSE GitHub
cropd.set_active_crop("potato", "Potato_701") #Potato
#cropd.set_active_crop("millet", "Millet_VanHeemst_1988") #Millet (Hirse)
#cropd.set_active_crop("wheat", "Winter_wheat_101") #Wheat
print(cropd)

"""load dummy soil file https://pcse.readthedocs.io/en/stable/code.html#pcse.util.DummySoilDataProvider"""
soild = DummySoilDataProvider()

"""load some standard site; https://pcse.readthedocs.io/en/stable/code.html#wofost72sitedataprovider"""
sited = WOFOST72SiteDataProvider(WAV=10) #WAV = initial soil moisture

"""put everything together with ParameterProvider"""
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

"""agro file"""
agromanagement_file =("data/agro/potato_agro_test.yaml") #insert path
agromanagement = YAMLAgroManagementReader(agromanagement_file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/crop/potato.yaml'

running the model with the given files

In [ ]:
#running the model
from pcse.models import Wofost72_WLP_CWB
wofsim = Wofost72_WLP_CWB(parameters, weatherdataprovider, agromanagement)

wofsim.run(days=10)
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

NameError: name 'parameters' is not defined

Visualization

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12,12), sharex=True)
for var, ax in zip(["DVS", "TAGP", "LAI", "SM"], axes.flatten()):
    ax.plot(df_results.index, df_results[var], 'b-')
    ax.set_title(var)
plt.show()
fig.autofmt_xdate()